In [1]:
!pip install pandas
!pip install torch


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2955 sha256=cba2f39a7e6cbe3dca591c7bdd36b32f747487a493d4a91df8df34d6826a5d62
  Stored in directory: /root/.cache/pip/wheels/f8/e0/3d/9d0c2020c44a519b9f02ab4fa6d2a4a996c98d79ab2f569fa1
Successfully built sklearn


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
from pathlib import Path

# create an empty dataframe with 'imgs' column
df = pd.DataFrame(columns=['imgs'])
# we will store the image files and captions here before putting it into dataframe
imgs, captions = [], []
# directory where the dataset is present
root_dir = Path("/content/drive/MyDrive/PureCode/D1")

In [5]:
import os
images=os.listdir(os.path.join(root_dir, "IMAGES"))[:200]

In [6]:
for img in images:
      imgs.append(os.path.join(root_dir, "IMAGES", img))
      annotation_file=Path(img).stem+ ".gui"
      with open(os.path.join(root_dir, "TEXT_LABELS", annotation_file), "r") as f:
           content = f.readlines()
           content=(" ".join(content)).strip()
           captions.append(content)





In [16]:
len(imgs)

200

In [7]:
df.loc[:, 'imgs'] = imgs
df.loc[:, 'captions'] = captions

In [8]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.9 MB/s eta 0:00:00


In [9]:
from transformers import AutoFeatureExtractor, AutoTokenizer

encoder_checkpoint = "google/vit-base-patch16-224-in21k"
decoder_checkpoint = "gpt2"

feature_extractor = AutoFeatureExtractor.from_pretrained(encoder_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(decoder_checkpoint)

/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [10]:
tokenizer.pad_token = tokenizer.eos_token


In [11]:
!pip install pillow


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
from PIL import Image

# maximum length for the captions
max_length = 350 #128
sample = df.iloc[0]

# sample image
image = Image.open(sample['imgs']).convert('RGB')
# sample caption
caption = sample['captions']

# apply feature extractor on the sample image
inputs = feature_extractor(images=image, return_tensors='pt')
# apply tokenizer
outputs = tokenizer(
            caption, 
            max_length=max_length, 
            truncation=True, 
            padding='max_length',
            return_tensors='pt',
        )

In [13]:
from torch.utils.data import Dataset

class LoadDataset(Dataset):
    def __init__(self, df):
        self.images = df['imgs'].values
        self.captions = df['captions'].values
    
    def __getitem__(self, idx):
        # everything to return is stored inside this dict
        inputs = dict()

        # load the image and apply feature_extractor
        image_path = str(self.images[idx])
        image = Image.open(image_path).convert("RGB")
        image = feature_extractor(images=image, return_tensors='pt')

        # load the caption and apply tokenizer
        caption = self.captions[idx]
        labels = tokenizer(
            caption, 
            max_length=max_length, 
            truncation=True, 
            padding='max_length',
            return_tensors='pt',
        )['input_ids'][0]
        
        # store the inputs(pixel_values) and labels(input_ids) in the dict we created
        inputs['pixel_values'] = image['pixel_values'].squeeze()   
        inputs['labels'] = labels
        return inputs
    
    def __len__(self):
        return len(self.images)

In [14]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

In [15]:
train_ds = LoadDataset(train_df)
test_ds = LoadDataset(test_df)

In [16]:
from transformers import VisionEncoderDecoderModel

model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    encoder_checkpoint, 
    decoder_checkpoint
)

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.2.crossattention.q_attn.weight', 'h.8.crossattention.c_attn.weight', 'h.11.crossattention.bias', 'h.9.crossattention.c_proj.weight', 'h.0.crossattention.q_attn.weight', 'h.4.crossattention.masked_bias', 'h.11.crossattention.c_attn.weight', 'h.2.crossattention.c_attn.weight', 'h.5.crossattention.q_attn.weight', 'h.5.crossattention.c_proj.weight', 'h.6.ln_cross_attn.weight', 'h.10.crossattention.c_attn.weight', 'h.5.ln_cross_attn.weight', 'h.9.crossattention.c_attn.weight', 'h.11.crossattention.q_attn.weight', 'h.11.crossattention.c_proj.bias', 'h.0.crossattention.c_attn.weight', 'h.5.crossattention.bias', 'h.10.crossattention.c_proj.bias', 'h.7.crossattention.bias', 'h.1.crossattention.c_attn.weight', 'h.0.crossattention.c_proj.bias', 'h.7.crossattention.c_proj.weight', 'h.0.ln_cross_attn.weight', 'h.1.crossattention.masked_bias', 'h.5.crossattention.c_attn.weight', 'h.7

In [17]:
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

In [18]:
num_beams = 4
model.config.num_beams = num_beams

In [19]:
from transformers import Seq2SeqTrainingArguments

# batch size
bs = 2

training_args = Seq2SeqTrainingArguments(
    output_dir="image-caption-generator", # name of the directory to store training outputs
    evaluation_strategy="epoch",          # evaluate after each epoch
    per_device_train_batch_size=bs,       # batch size during training
    per_device_eval_batch_size=bs,        # batch size during evaluation
    learning_rate=5e-5,
    weight_decay=0.01,                    # weight decay parameter for AdamW optimizer
    num_train_epochs=50,                   # number of epochs to train
    save_strategy='epoch',                # save checkpoints after each epoch
    report_to='none',                     # prevent reporting to wandb, mlflow...
)

In [20]:
from transformers import Seq2SeqTrainer, default_data_collator

trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=feature_extractor,
    data_collator=default_data_collator,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    args=training_args,
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.043292
2,No log,0.041955
3,No log,0.036483
4,No log,0.037038
5,No log,0.036823
6,No log,0.037554
7,0.085100,0.037087
8,0.085100,0.035657
9,0.085100,0.033724
10,0.085100,0.033909


TrainOutput(global_step=4000, training_loss=0.03121483838558197, metrics={'train_runtime': 2929.962, 'train_samples_per_second': 2.73, 'train_steps_per_second': 1.365, 'total_flos': 1.443711128961024e+18, 'train_loss': 0.03121483838558197, 'epoch': 50.0})

In [26]:
#inputs = test_ds[0]['pixel_values']
image = Image.open("/content/drive/MyDrive/PureCode/D1/IMAGES/FECC1897-8B91-426B-96F3-29DA91E30721.png").convert('RGB')

inputs = feature_extractor(images=image, return_tensors='pt')['pixel_values'].squeeze()


In [30]:
import torch

model.eval()
with torch.no_grad():

    # generate caption for the image
    out = model.generate(
        inputs.unsqueeze(0).to('cuda'), # move inputs to GPU
        num_beams=num_beams, 
        )

# convert token ids to string format
decoded_out = tokenizer.decode(out[0], skip_special_tokens=True)

print(decoded_out)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


header {
 btn-inactive, btn-inactive, btn-active


In [27]:
pd

<module 'pandas' from '/usr/local/lib/python3.9/dist-packages/pandas/__init__.py'>

In [29]:
df['captions'][1]

'header {\n btn-active, btn-inactive\n }\n row {\n quadruple {\n small-title, text, btn-green\n }\n quadruple {\n small-title, text, btn-green\n }\n quadruple {\n small-title, text, btn-red\n }\n quadruple {\n small-title, text, btn-orange\n }\n }\n row {\n single {\n small-title, text, btn-orange\n }\n }\n row {\n double {\n small-title, text, btn-green\n }\n double {\n small-title, text, btn-red\n }\n }'